In [27]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.power import TTestIndPower
from read_db.CH import Getch

In [28]:
# импортируем данные за период с 29.11 по 05.12
data = Getch("select user_id, exp_group, \
countIf(action = 'view') as views, \
countIf(action = 'like') as likes, \
likes / views as ctr \
from simulator.feed_actions \
where toDate(time) >= '2021-11-29' and toDate(time) <= '2021-12-05' and \
exp_group in (0, 1) \
group by user_id, exp_group").df

In [29]:
data.head()

,user_id,exp_group,views,likes,ctr
0,26117,1,21,5,0.238095
1,138232,1,29,5,0.172414
2,49204,1,62,10,0.161290
3,49286,1,19,7,0.368421
4,145957,1,129,54,0.418605


In [30]:
mu_control = data[data.exp_group == 0].ctr.mean() # среднее значение ctr в контрольной группе
mu_experiment = data[data.exp_group == 1].ctr.mean() # среднее значение ctr в экспериментальной группе
mu_control, mu_experiment

(0.22195492956578272, 0.23648646948093194)

In [31]:
sigma_control = data[data.exp_group == 0].ctr.std() # стандартное октлонение в контрольной группе
sigma_experiment = data[data.exp_group == 1].ctr.std() # стандартное отклонение в экспериментальной группе
sigma_control, sigma_experiment

(0.08787038294959033, 0.09014644584020075)

In [32]:
# посчитаем, чему оказался размера эффекта в проведенном A/B тесте
effect_data_test = (mu_experiment - mu_control) / ((sigma_control**2 + sigma_experiment**2) / 2) ** 0.5
print('Размер эффекта в проведенном АB/тесте = {0}'.format(round(effect_data_test,2) ))

Размер эффекта в проведенном АB/тесте = 0.16


In [33]:
# посмотрим, какой необходимый размер выборки нам потребуется в дальнейшем, если мы:
# 1. Хотим использовать t-test для сравнения CTR по юзерам.
# 2. Ожидаем, что размер эффекта равен 0.3.
# 3. Хотим, чтобы мощность теста равнялась 0.8.
alpha = 0.05
power = 0.8
effect = 0.3
analysis = TTestIndPower()
result = analysis.solve_power(effect, power = power, nobs1 = None, alpha = alpha)
print('Размер выборки (при условиях: мощность = {0}, размер эффекта = {1}) составляет {2} пользователей'.format(power, round(effect,3), round(result,3) ))

Размер выборки (при условиях: мощность = 0.8, размер эффекта = 0.3) составляет 175.385 пользователей


In [34]:
# посмотрим, каким должен быть размер выборки для проведенного нами A/B теста при сохранении мощности = 0.8
alpha = 0.05
power = 0.8
effect = effect_data_test
analysis = TTestIndPower()
result_test = analysis.solve_power(effect, power = power, nobs1 = None, alpha = alpha)
print('Размер выборки, необходимый для проведенного нами АB-теста (при условиях: мощность = {0}, размер эффекта = {1}) составляет {2} пользователей'.format(power, round(effect,2), round(result_test,3) ))

Размер выборки, необходимый для проведенного нами АB-теста (при условиях: мощность = 0.8, размер эффекта = 0.16) составляет 590.004 пользователей


### Вывод
- Размер эффекта в проведенном A/B тесте - 0.16;
- Минимальный размер выборки, для проведенного нами A/B теста - 590 пользователей, если хотим сохранить текущую мощность теста в 0.8.